# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# 🥭 Wallet [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Wallet.ipynb)

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

This notebook holds all the code around handling a wallet of private and public keys.

**Please be careful with your private keys, and don't provide them to any code if you are not 100% confident you know what it will do.**

In [ ]:
import json
import typing

from solana.account import Account
from solana.publickey import PublicKey

from Context import AccountInfo, Context


## Wallet class

The `Wallet` class wraps our understanding of saving and loading keys, and creating the appropriate Solana `Account` object.

To load a private key from a file, the file must be a JSON-formatted text file with a root array of the 64 bytes making up the secret key.

For example:
```
[200,48,184,13... for another 60 bytes...]
```
**TODO:** It would be good to be able to load a `Wallet` from a mnemonic string. I haven't yet found a Python library that can generate a BIP44 derived seed for Solana that matches the derived seeds created by Sollet and Ledger.

In [ ]:
class Wallet:
    def __init__(self, secret_key):
        self.secret_key = secret_key[0:32]
        self.account = Account(self.secret_key)

    @property
    def address(self) -> PublicKey:
        return self.account.public_key()

    def fetch_largest_token_account(self, context: Context, token_mint: PublicKey) -> typing.Optional[AccountInfo]:
        largest_account = context.fetch_largest_token_account_for_owner(self.address, token_mint)
        return largest_account

    @staticmethod
    def load(filename: str = "id.json") -> "Wallet":
        with open(filename) as json_file:
            data = json.load(json_file)
            return Wallet(data)


# Running

A simple harness to load a `Wallet` if the `wallet.json` file exists, and print out its `PublicKey`.

**Please be careful with your private keys!**

In [ ]:
if __name__ == "__main__":
    import os.path
    from Context import default_context

    filename = "id.json"

    if not os.path.isfile(filename):
        print(f"Wallet file '{filename}' is not present.")
    else:
        wallet = Wallet.load(filename)
        print("Wallet address:", wallet.address)
        # SRMuApVNdxXokk5GT7XD5cUUgXMBCoAz2LHeuAoKWRt is Serum's mint address
        print(wallet.fetch_largest_token_account(default_context, PublicKey("SRMuApVNdxXokk5GT7XD5cUUgXMBCoAz2LHeuAoKWRt")))
